# FiftyOne + RoCoLe + PADIM

This notebook demonstrates anomaly detection on coffee leaves using:
1. **FiftyOne** for dataset management and visualization
2. **ResNetR18** for feature extraction
3. **PaDiM** (Patch Distribution Modeling) for lightweight anomaly detection

The goal is to detect diseased coffee leaves by training only on healthy samples.

## 1. Setup and Dependencies

In [ ]:
# Install required packages if needed
# !pip install fiftyone torch torchvision scikit-learn tqdm numpy pillow scipy

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import fiftyone as fo
import fiftyone.brain as fob
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis
from tqdm.notebook import tqdm
import gc
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"FiftyOne version: {fo.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Import Coffee Leaves Dataset

Load the coffee leaves dataset from FiftyOne. This dataset contains:
- **Normal leaves**: Healthy coffee leaves (791 samples)
- **Anormal leaves**: Diseased coffee leaves (1538 samples)

In [ ]:
# Load the coffee leaves dataset
# If not already loaded, you may need to run the loading script first
dataset_name = "coffee_leaves"

if dataset_name in fo.list_datasets():
    dataset = fo.load_dataset(dataset_name)
    print(f"✅ Dataset '{dataset_name}' loaded successfully")
else:
    print(f"⚠️ Dataset '{dataset_name}' not found. Loading from Hugging Face...")
    from fiftyone.utils.huggingface import load_from_hub
    dataset = load_from_hub(
        "pjramg/Coffee_leaves_rocole_FO",
        name=dataset_name
    )

print(f"\nDataset info:")
print(f"  • Name: {dataset.name}")
print(f"  • Samples: {len(dataset)}")
print(f"  • Media type: {dataset.media_type}")

In [ ]:
# Analyze labels in the dataset
label_counts = {}
for sample in dataset.iter_samples(progress=False):
    label = "unknown"
    if hasattr(sample, 'ground_truth') and sample.ground_truth:
        if hasattr(sample.ground_truth, 'label'):
            label = sample.ground_truth.label
        elif hasattr(sample.ground_truth, 'detections') and sample.ground_truth.detections:
            if sample.ground_truth.detections:
                label = sample.ground_truth.detections[0].label
    
    label_counts[label] = label_counts.get(label, 0) + 1

print("Label distribution:")
for label, count in sorted(label_counts.items()):
    print(f"  • {label}: {count} samples")

# Visualize label distribution
plt.figure(figsize=(8, 4))
plt.bar(label_counts.keys(), label_counts.values(), color=['green', 'red'])
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Coffee Leaves Dataset Distribution')
plt.show()

## 3. Configure PaDiM Model

PaDiM (Patch Distribution Modeling) is a lightweight anomaly detection method that:
- Uses pretrained CNN features (no training required)
- Models the distribution of normal samples
- Detects anomalies based on distance from normal distribution

In [ ]:
# Configuration
BACKBONE = "resnet18"  # Lightweight backbone
LAYER = "layer3"       # Single layer for efficiency
IMAGE_SIZE = 224       # Standard ImageNet size
BATCH_SIZE = 50        # Process in batches to save memory

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pretrained backbone
print(f"\nLoading {BACKBONE} backbone...")
model = models.resnet18(weights='IMAGENET1K_V1')
model = model.to(device)
model.eval()
print("✅ Model loaded")

In [ ]:
# Image preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

# Hook for feature extraction
features = {}
def hook_fn(module, input, output):
    features['layer3'] = output

# Register hook on layer3
for name, module in model.named_modules():
    if name == LAYER:
        handle = module.register_forward_hook(hook_fn)
        print(f"✅ Hook registered on {name}")

## 4. Extract Features from Images

Extract deep features from all coffee leaf images using the pretrained ResNet18.
We process images in batches to manage memory efficiently.

In [ ]:
def extract_features_batch(filepaths):
    """Extract features for a batch of images"""
    batch_features = []
    
    for filepath in filepaths:
        try:
            # Load and preprocess image
            img = Image.open(filepath).convert('RGB')
            img_tensor = transform(img).unsqueeze(0).to(device)
            
            # Extract features
            with torch.no_grad():
                _ = model(img_tensor)
            
            # Get features from hook
            feat = features['layer3']
            
            # Global average pooling to reduce dimensionality
            feat = torch.nn.functional.adaptive_avg_pool2d(feat, (1, 1))
            feat = feat.cpu().numpy().flatten()
            batch_features.append(feat)
            
        except Exception as e:
            # Handle missing files gracefully
            batch_features.append(np.zeros(256))  # ResNet18 layer3 has 256 channels
    
    return np.array(batch_features)

In [ ]:
# Collect all filepaths and labels
print("Collecting dataset information...")
all_filepaths = []
all_labels = []
sample_ids = []

for sample in dataset.iter_samples(progress=False):
    label = "unknown"
    if hasattr(sample, 'ground_truth') and sample.ground_truth:
        if hasattr(sample.ground_truth, 'label'):
            label = sample.ground_truth.label
        elif hasattr(sample.ground_truth, 'detections') and sample.ground_truth.detections:
            if sample.ground_truth.detections:
                label = sample.ground_truth.detections[0].label
    
    all_filepaths.append(sample.filepath)
    all_labels.append(label)
    sample_ids.append(sample.id)

print(f"Collected {len(all_filepaths)} samples")

In [ ]:
# Extract features in batches
n_samples = len(all_filepaths)
n_batches = (n_samples + BATCH_SIZE - 1) // BATCH_SIZE
all_features = []

print(f"\nExtracting features from {n_samples} images in {n_batches} batches...")
print(f"Batch size: {BATCH_SIZE}")

for i in tqdm(range(0, n_samples, BATCH_SIZE), desc="Processing batches"):
    batch_files = all_filepaths[i:i+BATCH_SIZE]
    batch_features = extract_features_batch(batch_files)
    all_features.append(batch_features)
    
    # Clean up memory after each batch
    gc.collect()
    if device.type == 'cuda':
        torch.cuda.empty_cache()

# Concatenate all features
all_features = np.vstack(all_features)
print(f"\n✅ Extracted features shape: {all_features.shape}")

## 5. Train PaDiM Model

Train the PaDiM model by:
1. Fitting a Gaussian distribution on healthy (normal) samples
2. Computing anomaly scores based on distance from this distribution

In [ ]:
# Separate healthy and diseased samples
healthy_mask = np.array([(str(l).lower() == 'normal') for l in all_labels])
healthy_features = all_features[healthy_mask]
anomaly_features = all_features[~healthy_mask]

print(f"Training set (healthy): {len(healthy_features)} samples")
print(f"Test set (anomalies): {len(anomaly_features)} samples")

# Visualize feature distribution using first 2 principal components
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
features_2d = pca.fit_transform(all_features)

plt.figure(figsize=(10, 6))
plt.scatter(features_2d[healthy_mask, 0], features_2d[healthy_mask, 1], 
           c='green', alpha=0.5, label='Normal', s=10)
plt.scatter(features_2d[~healthy_mask, 0], features_2d[~healthy_mask, 1], 
           c='red', alpha=0.5, label='Anormal', s=10)
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('Feature Distribution (PCA Visualization)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
if len(healthy_features) > 10:  # Need minimum samples for training
    print("Training PaDiM model...")
    
    # Calculate mean and covariance of healthy distribution
    mean = np.mean(healthy_features, axis=0)
    
    # Calculate covariance with regularization for numerical stability
    try:
        cov = np.cov(healthy_features.T)
        # Add small value to diagonal for numerical stability
        cov += np.eye(cov.shape[0]) * 1e-5
        inv_cov = np.linalg.inv(cov)
        print("✅ Covariance matrix computed successfully")
        use_mahalanobis = True
    except Exception as e:
        print(f"⚠️ Using simplified Euclidean distance due to: {e}")
        inv_cov = np.eye(mean.shape[0])
        use_mahalanobis = False
    
    print(f"Model parameters:")
    print(f"  • Feature dimension: {mean.shape[0]}")
    print(f"  • Distance metric: {'Mahalanobis' if use_mahalanobis else 'Euclidean'}")
    
else:
    print("⚠️ Not enough healthy samples for training")

## 6. Calculate Anomaly Scores

In [ ]:
# Calculate anomaly scores for all samples
print("Calculating anomaly scores...")
anomaly_scores = []

for feat in tqdm(all_features, desc="Computing scores"):
    # Calculate distance from normal distribution
    diff = feat - mean
    
    if use_mahalanobis:
        # Mahalanobis distance
        score = np.sqrt(np.dot(np.dot(diff, inv_cov), diff.T))
    else:
        # Euclidean distance (more stable)
        score = np.sqrt(np.dot(diff, diff))
    
    anomaly_scores.append(float(score))

# Convert to numpy array
anomaly_scores = np.array(anomaly_scores)

# Normalize scores to [0, 1]
if anomaly_scores.max() > anomaly_scores.min():
    anomaly_scores = (anomaly_scores - anomaly_scores.min()) / (anomaly_scores.max() - anomaly_scores.min())

print(f"\n✅ Computed anomaly scores for {len(anomaly_scores)} samples")
print(f"Score statistics:")
print(f"  • Min: {anomaly_scores.min():.4f}")
print(f"  • Max: {anomaly_scores.max():.4f}")
print(f"  • Mean: {anomaly_scores.mean():.4f}")
print(f"  • Std: {anomaly_scores.std():.4f}")

In [ ]:
# Visualize anomaly score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of scores by class
axes[0].hist(anomaly_scores[healthy_mask], bins=30, alpha=0.6, 
            color='green', label='Normal', density=True)
axes[0].hist(anomaly_scores[~healthy_mask], bins=30, alpha=0.6, 
            color='red', label='Anormal', density=True)
axes[0].set_xlabel('Anomaly Score')
axes[0].set_ylabel('Density')
axes[0].set_title('Anomaly Score Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot comparison
data_to_plot = [anomaly_scores[healthy_mask], anomaly_scores[~healthy_mask]]
box = axes[1].boxplot(data_to_plot, labels=['Normal', 'Anormal'], 
                      patch_artist=True)
box['boxes'][0].set_facecolor('green')
box['boxes'][1].set_facecolor('red')
axes[1].set_ylabel('Anomaly Score')
axes[1].set_title('Anomaly Score Comparison')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate separation metrics
normal_mean = anomaly_scores[healthy_mask].mean()
anormal_mean = anomaly_scores[~healthy_mask].mean()
print(f"\nClass separation:")
print(f"  • Normal mean score: {normal_mean:.4f}")
print(f"  • Anormal mean score: {anormal_mean:.4f}")
print(f"  • Separation: {anormal_mean - normal_mean:.4f}")

## 7. Compute Embeddings and Visualization

Create 2D embeddings for visualization in FiftyOne using multiple techniques.

In [ ]:
# Method 1: PCA-based embeddings
from sklearn.decomposition import PCA

print("Computing PCA embeddings...")
pca = PCA(n_components=2)
pca_embeddings = pca.fit_transform(all_features)
print(f"✅ PCA embeddings shape: {pca_embeddings.shape}")
print(f"   Explained variance: {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
# Method 2: UMAP-based embeddings (if available)
try:
    import umap
    print("Computing UMAP embeddings...")
    reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15)
    umap_embeddings = reducer.fit_transform(all_features)
    print(f"✅ UMAP embeddings shape: {umap_embeddings.shape}")
except ImportError:
    print("⚠️ UMAP not installed. Install with: pip install umap-learn")
    umap_embeddings = None

In [ ]:
# Method 3: Anomaly score-based circular embeddings
print("Computing anomaly-based embeddings...")
# Map anomaly scores to a circle for intuitive visualization
# Normal samples (low scores) near center, anomalies (high scores) on periphery
angles = anomaly_scores * 2 * np.pi
radii = anomaly_scores
anomaly_embeddings = np.column_stack([
    radii * np.cos(angles),
    radii * np.sin(angles)
])
print(f"✅ Anomaly embeddings shape: {anomaly_embeddings.shape}")

In [ ]:
# Visualize all embeddings
n_methods = 3 if umap_embeddings is not None else 2
fig, axes = plt.subplots(1, n_methods, figsize=(6*n_methods, 5))

# PCA visualization
axes[0].scatter(pca_embeddings[healthy_mask, 0], pca_embeddings[healthy_mask, 1], 
               c='green', alpha=0.5, label='Normal', s=10)
axes[0].scatter(pca_embeddings[~healthy_mask, 0], pca_embeddings[~healthy_mask, 1], 
               c='red', alpha=0.5, label='Anormal', s=10)
axes[0].set_title('PCA Embeddings')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Anomaly score visualization
axes[1].scatter(anomaly_embeddings[healthy_mask, 0], anomaly_embeddings[healthy_mask, 1], 
               c='green', alpha=0.5, label='Normal', s=10)
axes[1].scatter(anomaly_embeddings[~healthy_mask, 0], anomaly_embeddings[~healthy_mask, 1], 
               c='red', alpha=0.5, label='Anormal', s=10)
axes[1].set_title('Anomaly Score Embeddings (Circular)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# UMAP visualization (if available)
if umap_embeddings is not None:
    axes[2].scatter(umap_embeddings[healthy_mask, 0], umap_embeddings[healthy_mask, 1], 
                   c='green', alpha=0.5, label='Normal', s=10)
    axes[2].scatter(umap_embeddings[~healthy_mask, 0], umap_embeddings[~healthy_mask, 1], 
                   c='red', alpha=0.5, label='Anormal', s=10)
    axes[2].set_title('UMAP Embeddings')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Add Results to FiftyOne Dataset

In [ ]:
# Add anomaly scores to dataset
print("Adding PaDiM results to FiftyOne dataset...")

for sample_id, score in tqdm(zip(sample_ids, anomaly_scores), 
                             total=len(sample_ids), 
                             desc="Updating samples"):
    sample = dataset[sample_id]
    sample["padim_score"] = float(score)
    sample.save()

print("✅ Anomaly scores added to dataset")

In [ ]:
# Add PCA visualization to FiftyOne
print("\nAdding PCA visualization to FiftyOne...")
fob.compute_visualization(
    dataset,
    embeddings=pca_embeddings,
    brain_key="padim_pca",
    method="manual"
)
print("✅ Added 'padim_pca' visualization")

In [ ]:
# Add anomaly score visualization to FiftyOne
print("\nAdding anomaly score visualization to FiftyOne...")
fob.compute_visualization(
    dataset,
    embeddings=anomaly_embeddings,
    brain_key="padim_anomaly",
    method="manual"
)
print("✅ Added 'padim_anomaly' visualization")

In [ ]:
# Add UMAP visualization if computed
if umap_embeddings is not None:
    print("\nAdding UMAP visualization to FiftyOne...")
    fob.compute_visualization(
        dataset,
        embeddings=umap_embeddings,
        brain_key="padim_umap",
        method="manual"
    )
    print("✅ Added 'padim_umap' visualization")

## 10. Launch FiftyOne App

In [ ]:
# Launch FiftyOne app to explore results
print("Launching FiftyOne app...")
print("\nYou can now:")
print("  • Sort by 'padim_score' to find most anomalous leaves")
print("  • Use the embeddings panel to explore clusters")
print("  • Filter samples by score threshold")
print("  • Compare normal vs anormal samples visually")

session = fo.launch_app(dataset)

# Create a view of the most anomalous samples
anomalous_view = dataset.sort_by("padim_score", reverse=True).limit(100)
session.view = anomalous_view

print("\n✨ Top 100 most anomalous samples displayed in app")